In [1]:
from preprocessor import Preprocessor
from collector import Collector

In [2]:
preprocessor = Preprocessor()
collector = Collector()

In [3]:
from random import uniform

time_sec = 0
def gen_data():
    global time_sec
    time_sec += uniform(0, 0.5)
    return {
        'bpm_s': 120.0 + uniform(-30, 30) if uniform(0,1)>0.5 else None,
        'uc_s': (50 + uniform(-30, 40)) if uniform(0,1)>0.5 else None,
        'time_sec': time_sec
    }

In [81]:
for i in range(1):
    new_data = gen_data()
    print(new_data)
    collector.update_collector(new_data)
    bpm_raw, uterus_raw = collector.get_data()
    (bpm_filtered, uc_filtered, fs_bpm, fs_uterus) = preprocessor.filter_physiological_signals(bpm_raw, uterus_raw, fs_estimated=4.0)
    print(preprocessor.compute_metrics(bpm_filtered, uc_filtered, fs_bpm, fs_uterus, True))

{'bpm_s': None, 'uc_s': None, 'time_sec': 431.51078454154384}
{'stv': 11.18688575511956, 'ltv': 68.77039612074205, 'baseline_heart_rate': 118.20421108569596, 'accelerations': [], 'decelerations': [], 'contractions': [{'start': 8, 'end': 855, 'duration': 593.7498313419519, 'amplitude': 32.18329480805645}], 'stvs': array([7.16969823e+01, 2.73248141e+01, 9.58696298e+00, 1.97500616e+00,
       7.11037840e+00, 1.18039791e+01, 8.99245580e+00, 5.59909090e+00,
       2.40552966e+00, 3.53167286e-01, 4.55390473e+00, 1.09239996e+01,
       6.00470447e+00, 1.53236155e+01, 3.63072073e+01, 3.05495206e+01,
       6.86722229e+00, 1.40309047e+01, 2.25725996e+01, 1.26995083e+01,
       5.02595232e+00, 1.32438869e+01, 2.70510961e+00, 1.23207523e+01,
       1.37553306e+01, 1.11535164e+01, 7.55957018e+00, 9.02217677e+00,
       2.26907742e+01, 1.32010996e+01, 6.40464989e+00, 1.73235104e+01,
       1.91250000e+01, 1.16175169e+01, 9.86451398e+00, 2.69575282e+01,
       2.10863014e+01, 3.97395345e+00, 6.92461

In [79]:
collector.get_data()

(       time_sec       value
 0      0.553434  139.089198
 1      1.120291  128.708339
 2      2.043001  149.062525
 3      2.312428   93.328655
 4      3.138284   90.936230
 ..          ...         ...
 385  180.182723  142.124340
 386  180.662462   90.934768
 387  180.766912  144.492165
 388  180.925575  124.707147
 389  181.279911  123.840635
 
 [390 rows x 2 columns],
        time_sec      value
 0      0.827356  44.573893
 1      1.871239  38.361952
 2      2.043001  21.680629
 3      2.778919  38.975366
 4      3.599285  84.370861
 ..          ...        ...
 361  179.925736  71.149584
 362  180.662462  79.653223
 363  180.766912  70.896441
 364  181.279911  46.606796
 365  181.672069  21.162685
 
 [366 rows x 2 columns])